# Introduction à PyTorch

Dans ce notebook, nous allons couvrir les bases de [PyTorch](https://pytorch.org/).
PyTorch est une librairie de tenseurs, comme JAX, que nous avons utilisé plus tôt dans le cours.
Par rapport à JAX, qui a un écosystème très intéressant pour l'inférence Bayésienne, PyTorch a un écosystème de _deep learning_ beaucoup plus mature, stable et complet.
Nous utiliserons PyTorch pour l'apprentissage profound et les réseaux neuronaux dans le reste du cours.
Il y a plusieurs avantages à utiliser une telle librairie lorsqu'on travaille avec des réseaux neuronaux artificiels, par exemple:

- Calcul des dérivées via la rétropropagation
- Utilisation de GPUs
- Plusieurs blocs de base requis pour implémenter différents modèles
- Plusieurs modèles sont déjà implémentés et disponibles en ligne

Tout comme avec JAX, l'interface PyTorch n'est pas trop loin de celle de NumPy dans plusieurs cas.
Il y a tout de même certaines subtilités que nous allons couvrir plus bas.

## Installation

Si ce n'est pas fait, il faut installer PyTorch. Rendez-vous sur [le site web](https://pytorch.org/). Plus bas sur la page il y a une section "Install PyTorch".
Sélectionnez les options et copiez la commande fournie à côté de "_Run this command_".
À moins que votre ordinateur n'ait un GPU pour le machine learning, cochez "CPU" dans _Compute Platform_.
Sur [Google Colab](https://colab.research.google.com/), vous pouvez utiliser CUDA 12.4 (voir [ce tutoriel](https://pytorch.org/tutorials/beginner/colab.html) pour plus de détails).

## Tenseurs

Habituellement, nous manipulons des ensembles de données via des tableaux NumPy (`np.array`).
Avec PyTorch, la structure équivalente est une _tenseur_ (`torch.tensor`)

### Création

On peut créer les tenseur à partir de listes imbriquées.

In [ ]:
import torch

data = [[1, 2], [3, 3]]
data_tensor = torch.tensor(data)
print(data_tensor)

Ici, les données sont copiées vers le tenseur: modifier la liste ne modifiera pas le tenseur et vice-versa.

On peut également créer notre tenseur à partir d'un tableau NumPy en utilisant `torch.from_numpy()`.

In [ ]:
import numpy as np

data_array = np.array(data)
data_tensor_np = torch.from_numpy(data_array)
print(data_tensor_np)

Quand on crée le tenseur à partir de NumPy, **les objets sont liés en mémoire**.
Ceci veut dire qu'une modification au tableau NumPy affectera aussi le tenseur PyTorch.

On peut créer une copie pour que les deux objets ne soient plus liés.
`torch.Tensor.clone()` est l'équivalent de `np.ndarray.copy()`.

In [ ]:
data_tensor_cloned = data_tensor_np.clone()
data_array[0, 0] = 100
print(data_array)  # modifié
print(data_tensor_np)  # modifié
print(data_tensor_cloned)  # original

La majorité des options pour créer un tableau NumPy sont répliquées par PyTorch. Que ce soit avec des constantes ou des nombres aléatoires.

In [ ]:
zeros = torch.zeros(2, 3)
ones = torch.ones_like(zeros)
rand_vals = torch.rand_like(ones)  # Uniforme [0, 1)
rand_norm = torch.randn_like(ones)  # Distribution normale
rand_ints = torch.randint_like(ones, 10)  # Entiers [0, 10)

print("Zeros:", zeros)
print("Zeros shape:", zeros.shape)
print("Ones:", ones)
print("Uniforme:", rand_vals)
print("Normal:", rand_norm)
print("Entiers:", rand_ints)

### Opérations

Les opérations mathématiques sont effectuées de manière similaire à NumPy: élément par élément.

In [ ]:
print(ones * 5)
print(rand_norm * rand_ints)

Les opérations logiques fonctionnent également, mais il faut convertir vers un type `bool`:

In [ ]:
# Erreur:
try:
    print(ones | zeros)
except Exception as e:
    print(f"Erreur détectée: {e}")

# On peut changer le type
print(ones.to(torch.bool) | zeros.to(torch.bool))
print(ones.to(torch.bool) & zeros.to(torch.bool))

On peut aussi convertir un tensor vers un tableau NumPy

In [ ]:
print(type(rand_vals))
print(rand_vals.numpy())
print(type(rand_vals.numpy()))

### Matplotlib

Les tenseurs PyTorch sont directement compatibles avec Matplotlib.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("tableau-colorblind10")

x = torch.arange(10)
y = 3 * x + 1
plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Graphique avec PyTorch")
plt.show()

### Utilisation de GPUs

Comme mentionné plus haut, un des principaux avantages des tenseurs PyTorch est leur compatibilité avec les GPUs,
qui permettent d'accélérer les calculs grâce à la parallélisation.
Si comme moi votre ordinateur n'a pas accès à un GPU, la cellule suivante affichera "`Pas de GPU :(`".

In [ ]:
# Si on avait un GPU, on pourrait envoyer le tenseur sur les GPU
if torch.cuda.is_available():
    print("Copié sur CUDA")
    x_data_gpu = data_tensor.to("cuda")
else:
    print("Pas de GPU :(")

Faire cette petite opération pour chaque tenseur n'est pas très pratique...
Une façon simple de ne vérifier qu'une seule fois la disponibilité d'un GPU est de créer une variable `device` qui garde cette information.
Ensuite, on peut créer les tenseurs en leur assignant une _device_.

In [ ]:
# On peut spécifier une seule fois et utiliser ensuite
device = "cuda" if torch.cuda.is_available() else "cpu"

# On peut spécifier le GPU à la création au lieu d'utiliser `.to()`
x_with_device = torch.rand(2, 2, device=device)
# Ou encore envoyer un tenseur existant
data_tensor.to(device)

print(x_with_device)
print(x_with_device.device)
print(x_with_device.is_cuda)

## Autodérivation avec `torch.autograd`

Comme nous avons avec JAX, un avantage des libraires de tenseur est le calcul automatique des gradients via l'autodérivation.
Pour l'apprentissage profond, cette fonctionnalité est essentielle lors de l'entraînement.
L'autodérivation et la rétropropagation sont disponibles dans le module `autograd` de PyTorch.

### Suivre les gradients avec `requires_grad` et `grad_fn`

Par défaut, un tenseur créé avec directement avec `torch` ne gardera pas de trace des gradients.

In [ ]:
x_nograd = torch.linspace(0.0, 2 * torch.pi, steps=25)  # pas de gradient
print("x_nograd:", x_nograd)
print("Requires grad:", x_nograd.requires_grad)

Il faut activer `autograd` explicitement avec `requires_grad=True`.

In [ ]:
x = torch.linspace(0.0, 1.0, steps=50, requires_grad=True)
print("x:", x)
print("Requires grad:", x.requires_grad)

Ensuite, toutes les opérations effectuées sur `x` tracerons le gradient.

In [ ]:
phase = 2 * torch.pi * x
print("phase:", phase)
print("phase.grad_fn:", phase.grad_fn)

On voit ici que PyTorch a enregistré que la fonction donnant le gradient pour `y` est une multiplication.
La même chose se produit pour de nouvelles opérations

In [ ]:
y = torch.sin(phase)
print("y:", y)
print("y.grad_fn:", y.grad_fn)

Remarquez qu'ici seule la dernière opération est affichée.
Il est cependant possible de remonter la chaîne des opérations:

In [ ]:
print("y.grad_fn:", y.grad_fn)
print("y.grad_fn.next_functions:", y.grad_fn.next_functions)
print("Next encore:", y.grad_fn.next_functions[0][0].next_functions)
print("Next encore (2x):", y.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)

Lorsqu'on atteint `AccumulatedGrad`, la chaîne est terminée (nous sommes revenus à `x`).

Même les opérations qui ne sont pas directement de l'arithmétique, par exemple une copie avec `x.clone()`, peuvent être suivies par `autograd`.

In [ ]:
print(y.clone())

Par contre, les tenseurs qui utilisent `requires_grad` ne sont pas compatible avec Matplotlib et NumPy...

In [ ]:
try:
    plt.plot(x, y)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Graphique avec PyTorch")
    plt.show()
except Exception as e:
    print("Oups...", e)

Pour régler ce problème, il suffit de "détacher" les gradients.
Cette fonction retourne un nouveau tenseur avec les mêmes valeurs, mais sans `requires_grad`.

In [ ]:
print(y.detach())

In [ ]:
plt.plot(x.detach(), y.detach())
plt.xlabel("x")
plt.ylabel("y")
plt.title("Graphique avec PyTorch")
plt.show()

### Calcul des gradients

Jusqu'à maintenant, nous avons emmagasiné les gradients via `requries_grad` et `grad_fn`, mais nous n'avons jamais demandé à PyTorch de calculer les gradients.
Ainsi, l'attribut `grad` de nos tenseurs n'est pas défini.

In [ ]:
print("x.grad:", x.grad)

Pour calculer les gradients, il suffit de prendre dernière sortie de notre chaîne d'opérations (ici `y`) et d'utiliser `.backward()`.
Cette fonction exécutera la rétropropagation.

In [ ]:
y.backward(torch.ones_like(y))  # Il faut passer le gradient initial quand y n'est pas scalaire, soit 1 ici (y vs y).

Les gradients pour les "feuilles" de notre graphe de calcul sont ensuite accessible. Pour obtenir $\frac{dy}{dx}$, on utilise donc `x.grad`:

In [ ]:
print("x.grad:", x.grad)

Par défaut, les gradients des étapes intermédiaires ne sont pas calculés:

In [ ]:
print("phase.grad", phase.grad)

Nous n'en aurons pas besoin, donc peut simplement ignorer le message d'avertissement.

**Exercice: affichez sur un graphique y en fonction de x ainsi que la dérivée $\frac{dy}{dx}$**.

In [ ]:
# TODO: Afficher le graphique

## Créer un modèle PyTorch (Régression linéaire)

Une des fonctionnalités très utile de PyTorch est son interface de modélisation `torch.nn`.
Elle nous donne plusieurs blocs et fonctions utiles pour construire des réseaux neuronaux.
Par contre, elle utilise la programmation orientée objet, donc la syntaxe est un peu différente des fonctions que nous utilisons habituellement.

Une façon simple de se familiariser avec l'interface est d'implémenter une régression linéaire.

### Données simulées

Comme d'habitude, commençons par simuler des données

In [ ]:
N = 25
noise_scale = 1.0
w_true, b_true = 2.0, -1.0
x = torch.linspace(-5, 5, steps=N)
y_true = w_true * x + b_true
y = y_true + noise_scale * torch.randn(N)

plt.plot(x, y, "kx", label="Données simulées")
plt.plot(x, y_true, label="Vrai signal", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

### Définition du modèle

Pour définir le modèle, il faut:

- Créer une classe qui hérite de `nn.Module`, ici `LinearModel`.
- Créer une méthode `__init__()`:
    - C'est cette méthode qui sera appelée lorsqu'on crée un modèle avec `model = LinearModel()`
    - La méthode `__init__` appelle la méthode `__init__()` de sa classe parent. On exécute ainsi le code que PyTorch implémente dans `nn.Module`.
    - C'est généralement ici que l'on définira les couches de notre réseau
- Créer une méthode `forward`. Cette méthode prend $x$ en entrée et exécute toute les couches de notre modèle.

**Rappel: `self` désigne l'objet lui-même et permet d'assigner et d'accéder aux attributs de notre classe.**

Dans notre cas, nous avons un modèle linéaire simple. Il faut donc une seule couche linéaire.
On pourrait l'implémenter manuellement, mais PyTorch a un bloc `nn.Linear` qui définit exactement ce dont nous avons besoin!
Selon [la documentation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html), `nn.Linear()` exécute
$$
y = x A^T + b.
$$
Tout ce que nous avons à faire, c'est de donner les dimensions d'entrée et de sortie. Les paramètre requis (poids et biais) seront créés automatiquement.
Dans notre cas nous avons $f : \mathbb{R} \rightarrow \mathbb{R}$, donc les dimensions sont de 1, le modèle sera donc équivalent à `y = a * x + b` où `a` et `b` sont des scalaires.
Pour inclure une couche linéaire dans notre modèle, il suffit donc de 1) créer cette couche dans `__init__()` et 2) exécuter cette couche dans `forward()`.
Ici, `self.linear` veut simplement dire qu'on emmagasine la couche linéaire dans une variable qui pourra être réutilisée ailleurs dans notre classe.


In [ ]:
from torch import nn
class LinearModel(nn.Module):
    def __init__(self):
        # On appelle la fonction nn.Module.__init__()
        # ceci permet au code d'initialisation de PyTorch de s'exécuter
        super().__init__()
        # On crée une couche linéaire pour l'utiliser dans forward
        self.linear = nn.Linear(in_features=1, out_features=1)

    def forward(self, x):
        return self.linear(x)

model = LinearModel()

Voilà, notre modèle est créé!

Par défaut PyTorch initialise le modèle à des paramètres aléatoires.
On peut accéder à ces paramètres.

In [ ]:
import pprint
pprint.pprint(list(model.named_parameters()))

On peut ensuite appeler notre modèle sur un point $x$ quelconque.
PyTorch s'attend à recevoir un tenseur:

In [ ]:
model(torch.tensor([0.0]))

Si on veut passer plusieurs points à la fois, il faut leur donner le format `(npts, ndim)`.

In [ ]:
X_test = torch.tensor([[0.0, 1.0]]).T
print(X_test.shape)
model(X_test)

Il faut donc formatter nos données `x` si on veut les passer dans notre modèle.

In [ ]:
X = x.unsqueeze(1)  # Ajoute une dimension à la position ndim, donc à la 2e dimension ici
Y = y.unsqueeze(1)
ypred = model(X)
print(ypred)

In [ ]:
plt.plot(x, y, "kx", label="Données simulées")
plt.plot(x, y_true, label="Vrai signal", alpha=0.5)
plt.plot(x, ypred.detach(), label="Prédiction initiale", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

### Entraînement du modèle (optimisation)

Les méthodes d'optimisation PyTorch assez différentes de celles fournies par SciPy.
On les utilise typiquement de la façon suivante:

- Définition d'une fonction objectif (_loss_)
- Définition d'un optimiseur (ici `SGD`)
- On crée manuellement une boucle dans laquelle:
  - On calcule la fonction de objectif
  - On propage les gradients
  - On fait faire un pas à l'optimiseur
  - On remet les gradients à 0 avant la prochaine itération

C'est généralement une bonne idée d'emmagasiner ou d'afficher les valeurs de la fonction objectif pour vérifier son évolution.
Si elle varie encore beaucoup, on peut ajouter des itérations.

Ici, on utilise l'erreur carrée moyenne pour notre fonction objectif, soit

$$
\mathrm{MSE} = \frac{1}{N} \sum_i^N (\hat{y_i} - y_i)^2
$$

**Exercice: Complétez la fonction `loss_fn`. Assurez-vous d'utiliser `torch` pour calculer la moyenne.**

**Exercice: Créez un tableau vide de taille `niter` pour y placer les valeurs de `loss`. Affichez ces valeurs après l'entraînement pour vérifier la convergence.**

In [ ]:
model = LinearModel()

def loss_fn(y, ypred):
    # TODO: Loss function

# On donne les paramètres du modèle à SGD.
# L'optimiseur peut ensuite se servir de leur gradient.
optimizer = torch.optim.SGD(model.parameters())

niter = 2000

# TODO: Tableau loss_vals
for i in range(niter):

    # On calcule la prédiction du modèle
    # et la fonction objectif
    ypred = model(X)
    loss = loss_fn(Y, ypred)


    loss.backward()  # backprop
    optimizer.step()  # un pas dans l'espace-paramètre, utilise le gradietn calculé par loss.backward()
    optimizer.zero_grad()  # on réinitialise les gradients pour ne pas les acumuler entre les itérations

    # TODO: Ajouter loss à loss_vals

In [ ]:
# TODO: Graphique loss vs itération

Voilà, le modèle, est en entraîné!
Notez que normalement on utiliserait une partie des données comme ensemble de test et qu'on testerait la généralisation du modèle.
Nous en verrons un exemple au prochain cours.
Ici, nous allons nous contenter de cette optimisation simple.

### Prédictions avec le modèle entraîné

Maintenant que le modèle est entraîné, vérifions la valeur des paramètres la prédiction du modèle.

**Exercice: affichez la valeur des poids et un graphique montrant la prédiction du modèle.**

In [ ]:
# TODO: Imprimer les poids

In [ ]:
# TODO: Ajouter la prédiction optimisée au graphique
plt.plot(x, Y, "kx", label="Données simulées")
plt.plot(x, y_true, label="Vrai signal", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

## Exercice: XOR

Tel que vu en classe, l'opération XOR est une opération logique qui prend en entrée deux nombres ($x_1$, $x_2$) et qui retourne:

- $y=1$ si l'un des deux nombres est 1, mais pas les deux
- $y=0$ si les deux nombres sont 0 ou si les deux nombres sont 1

On peut représenter cette situation avec un vecteur $x = [x_1, x_2]$ en entrée et un scalaire $y$ en sortie.

Le graphique ci-dessous illustre le problème.

In [ ]:
X = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y = torch.tensor([0.0, 1.0, 1.0, 0.0])
Y = y.unsqueeze(1)

plt.scatter(X[:, 0], X[:, 1], c=y, s=100, marker='x', label="Données",cmap="coolwarm")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.axis("square")
plt.title("Données pour l'opération XOR")
plt.colorbar(label="y")
plt.legend()
plt.show()

Nous avons également vu en classe que l'opération XOR ne peut être modélisée par une régression linéaire, mais peut l'être par un petit réseau neuronal contenant une couche cachée de deux neurones.

### Régression linéaire

Commençons par vérifier que la régression linéaire se comporte comme nous avons vu en classe, c'est à dire qu'elle converge vers des poids à 0 et un biais à 0.5.

**Exercice: copiez le modèle de régression linéaire ci-dessus, mais adaptez le pour accepter deux dimensions d'entrée (`in_features`)**
**Initialisez ensuite le modèle et testez le sur les données `X`. Imprimez le tenseur retourné.**

In [ ]:
# TODO: Régression linéaire 2D vers 1D

**Imprimez les paramètres du modèle. Le format est-il celui auquel vous vous attendiez?**

In [ ]:
# TODO: Imprimer les paramètres

**Copiez la boucle d'entraînement de la section précédente, mais adaptez-la à ce nouveau problème. Assurez-vous d'utiliser `Y` et non `y` pour l'entraînement! Utilisez un nombre d'itération suffisant pour que la fonction objectif semble avoir convergé à une valeur stable.**

In [ ]:
# TODO: Entraînement

**Affichez maintenant les paramètres entraînés et la prédiction du modèle. Vous pouvez simplement l'imprimer ou la représenter sur un graphique.**

In [ ]:
# TODO: Paramètres et prédiction

La prédiction donne le résultat vu en classe.

### Réseau neuronal simple

Essayons maintenant un réseau neuronal avec une couche cachée de deux neurones et une activation sigmoide.

**Implémentez un réseau neuronal avec l'architecture suivante**:

- Dans la fonction `__init__()`, définissez:
    - Une couche linéaire avec 2 dimensions d'entrée ($x$) et deux dimensions de sortie ($z$)
    - Une fonction d'activation sigmoide (`nn.Sigmoid`) qui donnera $h = g(z)$. Vous pouvez la créer avec `self.activation = nn.Sigmoid()`
    - Une couche linéaire avec deux dimensions d'entrée ($h$) et une dimension de sortie ($y$)
- Dans la fonction `forward()`, appelez les couches dans l'ordre et retournez la valeur y finale.

In [ ]:
from torch import nn
class TinyNN(nn.Module):
    def __init__(self):
        # TODO: Fonction init
    def forward(self, x):
        # TODO: propagation entre les couches

**Initialisez le modèle et affichez ses paramètres**

In [ ]:
# TODO: Modèle et paramètres

**Optimisez le modèle en complétant la boucle d'optimisation ci-dessous. Affichez ensuite la fonction objectif en fonction du temps.**

Ici, l'optimiseur a été modifié pour augmenter le taux d'apprentissage (`lr` pour _learning rate_) et le momentum (que nous verrons dans les prochains cours).
La surface d'optimisation est plus complexe et ces paramètres aident à ce que le modèle converge.

In [ ]:
model = TinyNN()

optimizer = torch.optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

# TODO: Boucle d'entraînement

**Affichez maintenant les paramètres entraînés et la prédiction du modèle. Vous pouvez simplement l'imprimer ou la représenter sur un graphique.**

Si les résultats ne sont pas satisfaisants, retournez à l'étape d'entraînement et tentez d'ajouter des itérations!

In [ ]:
# TODO: Paramètres et prédiction

**Utilisez `model.linear1` et `model.activation` pour obtenir les valeurs $h$ de la couche cachées. Affichez les valeurs de $y$ en fonction de $h_1$ et $h_2$. Pourquoi cette représentation améliore-t-elle la performance du modèle?**


In [ ]:
# TODO: Afficher la représentation intermédiaire h